<a href="https://colab.research.google.com/github/MineCamci/My_Project_For_Data_Science/blob/main/CRM_Analysis_Customer_Lifetime_Value_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
########################################
# CUSTOMER LIFETIME VALUE
########################################

In [ ]:
#1.VERI HAZIRLAMA

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.float_format', lambda x: '%.5f' %x)

df_ = pd.read_csv("online_retail_09_10.csv")
df = df_.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25000,13085.00000,United Kingdom


In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     107927
Country             0
dtype: int64

In [ ]:
df = df[~df["InvoiceNo"].str.contains("C", na=False)]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25000,13085.00000,United Kingdom


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
UnitPrice,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
CustomerID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [ ]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.00000,13.58669,96.84223,1.00000,2.00000,5.00000,12.00000,19152.00000
UnitPrice,407695.00000,3.29419,34.75666,0.00000,1.25000,1.95000,3.75000,10953.50000
CustomerID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [ ]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25000,13085.00000,United Kingdom,30.00000


In [ ]:
cltv_c = df.groupby("CustomerID").agg({'InvoiceNo': lambda x: x.nunique(), #cltv metriği olan total_transaction erişmiş oluruz
                                       'Quantity': lambda x: x.sum(), #analiz etmek için istedik sadece
                                       'TotalPrice': lambda x: x.sum() }) # cltv metriği olan total_price erişmiş olacağız

In [ ]:
cltv_c

,InvoiceNo,Quantity,TotalPrice
CustomerID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [ ]:
cltv_c.columns

Index(['InvoiceNo', 'Quantity', 'TotalPrice'], dtype='object')

In [ ]:
cltv_c.columns = ["total_transaction", "total_unit", "total_price"] #rfm deki frequency total_transaction monetary total_price
cltv_c #tekilleştirdik 12346 numaralı müşterinin işlemleri tek satırda görünüyor artık

,total_transaction,total_unit,total_price
CustomerID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [ ]:
#2.Average Order Value (Ortalama Sipariş Değeri) (average_order_value = total_price / total_transaction)

In [ ]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
CustomerID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


In [ ]:
#3.Purchase Frequency (Satın Alma Sıklığı) = (total_transaction / total_number_of_customer)

#cltv_c["total_number_of_customer"] =
cltv_c.shape[0]
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
CustomerID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


In [ ]:
#4. Repeat Rate & Churn Rate (Tekrarlama Oranı ve Kaybetme Oranı) = (Birden Fazla Alışveriş Yapan Müşteri Sayısı / Tüm Müşteriler)
#Churn Rate = 1- Repeat Rate

In [ ]:
cltv_c[(cltv_c["total_transaction"] > 1)].shape[0] #Birden Fazla Alışveriş Yapan Müşteri Sayısı için shape dedik

2893

In [ ]:
#cltv_c["repeat_rate"] =
repeat_rate = cltv_c[(cltv_c["total_transaction"] > 1)].shape[0] / cltv_c.shape[0]
repeat_rate
#cltv_c

0.6706073249884098

In [ ]:
churn_rate = 1 - repeat_rate
churn_rate

0.3293926750115902

In [ ]:
#5. Profit Margin (Kar Marjı) = total_price * 0.10 (0.10 şirket tarafından belirlenmesi gereken sabit bir değer)

cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
CustomerID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


In [ ]:
#6. Customer Value (Müşteri Değeri) = (customer_value = average_order_value * purchase_frequency)
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c['purchase_frequency']
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
CustomerID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


In [ ]:
#7. Customer Lifetime Value (Müşteri Yaşam Boyu Değeri) = (CLTV = (customer_value / churn_rate) * profit_margin )

cltv_c["CLTV"] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,CLTV
CustomerID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


In [ ]:
cltv_c.sort_values(by = 'CLTV' , ascending = False).head() #küçükten büyüğe sırala

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,CLTV
CustomerID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [ ]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
average_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_frequency,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
CLTV,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


In [ ]:
#8. CREATING SEGMENTS (SEGMENTLERIN OLUSTURULMASI)

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,CLTV
CustomerID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


In [ ]:
cltv_c.sort_values(by = 'CLTV' , ascending = False).tail()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,CLTV
CustomerID,,,,,,,,
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061
14103.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000
14827.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000


In [ ]:
#Örneğin 4 gruba (segmente) ayırmak isteyelim birlikte analiz edelim
#qcut fonksiyonu ile yapabiliriz qcut der ki bana bir değişken ver , kaç parçaya bölmek istediğini söyle , label ları da verirsen ben sana bunu küçükten büyüğe sıralar istediğin şekilde etiketlendiririm

cltv_c["Segment"] = pd.qcut(cltv_c["CLTV"], 4, labels=["D","C","B","A"]) #%25 lik kısma D %50 lik kısma C %75 kısma B %100 kısma A olarak ayırdı

cltv_c.sort_values(by = 'CLTV' , ascending = False).head()


,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,CLTV,Segment
CustomerID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


In [ ]:
cltv_c.groupby("Segment").agg({"count","mean","sum"})

total_transaction                total_unit                   \
                      sum     mean count        sum       mean count   
Segment                                                                
D                    1326  1.22892  1079     117616  109.00463  1079   
C                    2160  2.00371  1078     305135  283.05659  1078   
B                    4063  3.76902  1078     733211  680.15863  1078   
A                   11666 10.81186  1079    4383262 4062.33735  1079   

          total_price                  average_order_value  ...  \
                  sum       mean count                 sum  ...   
Segment                                                     ...   
D        192265.13000  178.18826  1079        169805.46550  ...   
C        513016.45300  475.89652  1078        317422.35514  ...   
B       1219605.20000 1131.35918  1078        420127.12365  ...   
A       6907116.49100 6401.40546  1079        723972.22347  ...   

        purchase_frequency profit_margin                 customer_value  \
                     count           sum      mean count            sum   
Segment                                                                   
D                     1079   19226.51300  17.81883  1079       44.56772   
C                     1078   51301.64530  47.58965  1078      118.91897   
B                     1078  121960.52000 113.13592  1078      282.70867   
A                     1079  690711.64910 640.14055  1079     1601.09330   

                                CLTV                    
           mean count            sum        mean count  
Segment                                                 
D       0.04130  1079     2849.32790     2.64071  1079  
C       0.11031  1078    18184.26342    16.86852  1078  
B       0.26225  1078   103549.86329    96.05739  1078  
A       1.48387  1079 25257295.04118 23408.05843  1079  

[4 rows x 24 columns]

In [ ]:
cltv_c.to_csv("cltv_c.csv")